In [379]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [332]:

import chardet
path = '../LR2/spotify-2023.csv'
with open(path, 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

data = pd.read_csv(path, encoding=encoding)
data = data.drop(574)
data = data.dropna()
print(data.count());

track_name              816
artist(s)_name          816
artist_count            816
released_year           816
released_month          816
released_day            816
in_spotify_playlists    816
in_spotify_charts       816
streams                 816
in_apple_playlists      816
in_apple_charts         816
in_deezer_playlists     816
in_deezer_charts        816
in_shazam_charts        816
bpm                     816
key                     816
mode                    816
danceability_%          816
valence_%               816
energy_%                816
acousticness_%          816
instrumentalness_%      816
liveness_%              816
speechiness_%           816
dtype: int64


# Было решено предсказывать danceability

## Задание 2

### ввиду того, что предсказываем процент danceability, то очевдно, что это задача регрессии

In [252]:
selected_features = ['artist_count', 'released_year', 'streams', 'in_apple_playlists']
target_variable = 'danceability_%'
X = data[selected_features]
X['streams']=X['streams'].astype(float)
X['artist_count']=X['artist_count'].astype(float)
X['released_year']=X['released_year'].astype(float)
X['in_apple_playlists']=X['in_apple_playlists'].astype(float)
Y = data[target_variable]

/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_70383/1403072990.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['streams']=X['streams'].astype(float)
/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_70383/1403072990.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['artist_count']=X['artist_count'].astype(float)
/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_70383/1403072990.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [253]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=17)

In [254]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train[selected_features])

X_test_scaled = scaler.fit_transform(X_test[selected_features])

X.loc[X_train.index, selected_features] = X_train_scaled
X.loc[X_test.index, selected_features] = X_test_scaled




In [255]:
X.std()

artist_count          1.000613
released_year         1.000613
streams               1.000613
in_apple_playlists    1.000613
dtype: float64

In [256]:
X['streams'].mean()

-6.095342095981252e-17

# Задание 4, 5

In [338]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.linear_model import SGDRegressor

from sklearn.metrics import mean_absolute_error,  mean_squared_error



In [385]:
def getMetrics(test,predict):
    MAE  = mean_absolute_error(test, predict)
    print(f"MAE: {MAE:.2f}")

    RMSE = np.sqrt(mean_squared_error(test, predict))
    print(f"RMSE: {RMSE:.2f}")

    MAPE = np.mean(np.abs((test - predict) / test)) * 100
    print(f"MAPE: {MAPE:.2f}%")

## тест сет

In [386]:
linClassifier  = LinearRegression();

print(f"{'Тест сет':–^25}")


linClassifier.fit(X, Y)
lr_pred = linClassifier.predict(X_test_scaled)
getMetrics(y_test, lr_pred)

print(f"{'Трейн сет':–^25}")
linClassifier  = LinearRegression();
linClassifier.fit(X, Y)

logr_pred_train = linClassifier.predict(X_train_scaled)

getMetrics(y_train, logr_pred_train)


––––––––Тест сет–––––––––
MAE: 12.78
RMSE: 15.64
MAPE: 22.01%
––––––––Трейн сет––––––––
MAE: 10.97
RMSE: 13.56
MAPE: 18.82%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [387]:
logClassifier  = LogisticRegression()
logClassifier.fit(X, Y)
logr_pred = logClassifier.predict(X_test_scaled)

getMetrics(y_test, logr_pred)

logClassifier  = LogisticRegression()
logClassifier.fit(X, Y)
logr_pred = logClassifier.predict(X_train_scaled)
getMetrics(y_train, logr_pred)



MAE: 12.95
RMSE: 17.00
MAPE: 23.70%
MAE: 11.60
RMSE: 14.99
MAPE: 20.63%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [333]:
knn  = KNeighborsClassifier(n_neighbors=2)
knn.fit(X, Y)
knn_pred = knn.predict(X_test_scaled)
print(f"{'Тест сет':–^25}")


getMetrics(y_test, knn_pred)


print(f"{'Трейн сет':–^25}")


knn_pred = KNeighborsClassifier()
knn_pred.fit(X, Y)

knn_pred_train = knn_pred.predict(X_train_scaled)

getMetrics(y_train, knn_pred_train)


––––––––Тест сет–––––––––
MAE: 8.01
RMSE: 14.20
MAPE: 10.37%
––––––––Трейн сет––––––––
MAE: 14.57
RMSE: 19.95
MAPE: 20.44%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [328]:
knn_regr = KNeighborsRegressor(5)
knn_regr.fit(X,Y)
knn_regr_pred = knn_regr.predict(X_test_scaled)

print(f"{'Тест сет':–^25}")

getMetrics(y_test, knn_regr_pred)

print(f"{'Трейн сет':–^25}")

knn_regr = KNeighborsRegressor(5)
knn_regr.fit(X,Y)
knn_regr_pred = knn_regr.predict(X_test_scaled)

getMetrics(y_test, knn_regr_pred)



––––––––Тест сет–––––––––
MAE: 10.57
RMSE: 13.34
MAPE: 17.93%
––––––––Трейн сет––––––––
MAE: 10.57
RMSE: 13.34
MAPE: 17.93%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


In [326]:
treeClassifier  = DecisionTreeClassifier(criterion='entropy', max_depth=10)


treeClassifier.fit(X, Y)
tree_classifieur_pred = treeClassifier.predict(X_test_scaled)
print(f"{'Тест сет':–^25}")

getMetrics(y_test, tree_classifieur_pred)


print(f"{'Трейн сет':–^25}")

treeClassifier  = DecisionTreeClassifier(criterion='entropy', max_depth=10)


treeClassifier.fit(X, Y)
tree_classifieur_pred = treeClassifier.predict(X_train_scaled)

getMetrics(y_train, tree_classifieur_pred)


––––––––Тест сет–––––––––
MAE: 2.44
RMSE: 7.07
MAPE: 3.47%
––––––––Трейн сет––––––––
MAE: 3.04
RMSE: 8.07
MAPE: 4.31%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [325]:
treeRegr  = DecisionTreeRegressor(max_depth=10)


treeRegr.fit(X, Y)
treeRegr_pred = treeRegr.predict(X_test_scaled)
print(f"{'Тест сет':–^25}")

getMetrics(y_test, treeRegr_pred)

print(f"{'Трейн сет':–^25}")

treeRegr  = DecisionTreeRegressor(max_depth=10)


treeRegr.fit(X, Y)
treeRegr_pred = treeRegr.predict(X_train_scaled)

getMetrics(y_train, treeRegr_pred)



––––––––Тест сет–––––––––
MAE: 5.73
RMSE: 9.01
MAPE: 9.30%
––––––––Трейн сет––––––––
MAE: 5.55
RMSE: 8.55
MAPE: 8.96%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [270]:
forest_classifier  = RandomForestClassifier()


forest_classifier.fit(X, Y)
forest_classifier_pred = forest_classifier.predict(X_test_scaled)

getMetrics(y_test, forest_classifier_pred)
# Это странный результат...

MAE: 0.00
RMSE: 0.00
MAPE: 0.00%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [324]:
forestRegr  = RandomForestRegressor()


forestRegr.fit(X, Y)
forestRegr_pred = forestRegr.predict(X_test_scaled);
print(f"{'Тест сет':–^25}")

getMetrics(y_test, forestRegr_pred)


print(f"{'Трейн сет':–^25}")

forestRegr  = RandomForestRegressor()


forestRegr.fit(X, Y)
forestRegr_pred = forestRegr.predict(X_train_scaled);

getMetrics(y_train, forestRegr_pred)


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


––––––––Тест сет–––––––––
MAE: 4.63
RMSE: 5.96
MAPE: 7.93%
––––––––Трейн сет––––––––
MAE: 4.11
RMSE: 5.23
MAPE: 6.98%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


## Попытки улучшить модель

In [394]:
from sklearn.preprocessing import PolynomialFeatures


for power in range(1,8):
    poly = PolynomialFeatures(degree=power);

    X_poly = poly.fit_transform(X[selected_features])
    regr = LinearRegression()

    regr.fit(X_poly, Y)
    pred = regr.predict(poly.transform(X_test_scaled))
    print(f"{f'Степень {power}':–^25}")
    getMetrics(y_test, pred)

plt.show()

/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


––––––––Степень 1––––––––
MAE: 12.78
RMSE: 15.64
MAPE: 22.01%
––––––––Степень 2––––––––
MAE: 12.54
RMSE: 15.47
MAPE: 21.59%
––––––––Степень 3––––––––
MAE: 11.80
RMSE: 14.73
MAPE: 20.42%
––––––––Степень 4––––––––
MAE: 11.29
RMSE: 14.33
MAPE: 19.46%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


––––––––Степень 5––––––––
MAE: 10.56
RMSE: 13.63
MAPE: 18.26%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


––––––––Степень 6––––––––
MAE: 9.06
RMSE: 12.29
MAPE: 15.81%
––––––––Степень 7––––––––
MAE: 7.10
RMSE: 10.28
MAPE: 12.32%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
